In [35]:
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import pandas as pd
import time
import os
import pyodbc
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
import warnings
from skopt import BayesSearchCV
from sklearn.model_selection import GridSearchCV, LeaveOneOut
from sklearn.feature_selection import SelectFromModel
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import StratifiedKFold
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from sklearn.metrics import f1_score, accuracy_score
from random import randint
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from utils import *
from slugify import slugify
from datetime import date

In [36]:
server = 'localhost\SQLEXPRESS'
database = 'nba_game_data'

In [37]:
final_model = ''
final_model_cv_score = 0
final_model_score = 0
final_scoring_type = 'career'
final_dataset = -999
type_of_data = 'noraml_normal'
final_x_transformer = None
base_score = 0 
final_X = None
final_y = None
def model(player_id, pt_thresh, opp, over_or_under = 'over'):
    global final_X,final_y,final_model, final_model_cv_score, final_model_score, final_scoring_type, final_dataset, type_of_data, final_x_transformer, base_score
    if over_or_under == 'under':
        line_sign = '<'
    else:
        line_sign = '>'
    sql = f'''
    with base as (
    SELECT
           pgl.[SEASON_YEAR]
          ,pgl.[PLAYER_ID]
          ,pgl.[PLAYER_NAME]
          ,pgl.[NICKNAME]
          ,pgl.[TEAM_ID]
          ,pgl.[TEAM_ABBREVIATION]
          ,pgl.[TEAM_NAME]
          ,pgl.[GAME_ID]
          ,pgl.[GAME_DATE]
          ,LAG (pgl.[GAME_DATE]) OVER (PARTITION BY pgl.[PLAYER_ID] ORDER BY pgl.[GAME_DATE]) AS Last_Game_Played
          ,pgl.[MATCHUP]
          ,pgl.[WL]
          ,pgl.[yearSeason]
          ,CASE WHEN PGL.WL = 'W' THEN 1 ELSE 0 END AS WLInt
          ,LAG (CASE WHEN PGL.WL = 'W' THEN 1 ELSE 0 END) OVER (PARTITION BY pgl.[PLAYER_ID] ORDER BY pgl.[GAME_DATE]) AS did_they_win_last_game
         ,PTS
      FROM [nba_game_data].[dbo].[PlayerGameLogs] pgl


      WHERE pgl.PLAYER_ID = '{player_id}'
      )
      SELECT 
     [PLAYER_ID]
    ,[GAME_ID]
    ,[GAME_DATE]
    ,yearSeason
    ,[PTS]
    ,CASE WHEN [PTS] {line_sign} {pt_thresh} THEN 1 ELSE 0 END AS PtsThreshold
    ,SUM(WLInt) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN 11 PRECEDING AND 1 PRECEDING) AS RunningGamesWonLast10
    ,DATEDIFF(day, Last_Game_Played,[GAME_DATE])-1 AS daysRest
    ,did_they_win_last_game

      FROM base

      order by GAME_DATE

    '''
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
    cursor = cnxn.cursor()
    data = pd.read_sql(sql,cnxn)

    sql = f'''
    with base as (
    SELECT
           pgl.[SEASON_YEAR]
          ,pgl.[PLAYER_ID]
          ,pgl.[PLAYER_NAME]
          ,pgl.[NICKNAME]
          ,pgl.[TEAM_ID]
          ,pgl.[TEAM_ABBREVIATION]
          ,pgl.[TEAM_NAME]
          ,pgl.[GAME_ID]
          ,pgl.[GAME_DATE]
          ,LAG (pgl.[GAME_DATE]) OVER (PARTITION BY pgl.[PLAYER_ID] ORDER BY pgl.[GAME_DATE]) AS Last_Game_Played
          ,pgl.[MATCHUP]
          ,pgl.[WL]
          ,CASE WHEN PGL.WL = 'W' THEN 1 ELSE 0 END AS WLInt


      FROM [nba_game_data].[dbo].[PlayerGameLogs] pgl



      WHERE pgl.PLAYER_ID = '{player_id}'
      )
      SELECT 
     [PLAYER_ID]
    ,[GAME_ID]
    ,SUM(WLInt) OVER (PARTITION BY PLAYER_ID ORDER BY GAME_DATE ROWS BETWEEN 11 PRECEDING AND CURRENT ROW) AS RunningGamesWonLast10
    ,DATEDIFF(day, [GAME_DATE], GETDATE())-1 AS daysRest
    ,WLInt AS did_they_win_last_game

      FROM base

      order by GAME_DATE

    '''
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
    cursor = cnxn.cursor()
    pred = pd.read_sql(sql,cnxn)

    numbers = [2,5,10,20]

    for i in numbers:
        temp = get_player_last_n_data(player_id,i)
        temp = temp.drop(columns = ['TEAM_ABBREVIATION','oppAbrv'])
        data = pd.merge(data, temp, on=['PLAYER_ID', 'GAME_ID'])
        del temp

        temp = get_pred_data(player_id,i,opp)

        pred = pd.merge(pred, temp, on=['PLAYER_ID', 'GAME_ID'])
        del temp

    pred = pred.iloc[-1:].drop(columns=['PLAYER_ID','GAME_ID'])

    # Determine outliers using the IQR method
    Q1 = data['PTS'].quantile(0.25)
    Q3 = data['PTS'].quantile(0.75)
    IQR = Q3 - Q1

    # Define bounds for the outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Filter out outliers
    data = data[(data['PTS'] >= lower_bound) &
                                               (data['PTS'] <= upper_bound)]
    filtered = data.loc[(data['PTS'] < (pt_thresh - 3)) | (data['PTS'] > (pt_thresh + 3))]#['PTS']
    df_season_buffer = filtered.sort_values(by='GAME_DATE').tail(82)
    df_season_buffer = df_season_buffer.drop(columns=['PLAYER_ID','GAME_ID','GAME_DATE','PTS','yearSeason'])

    df_last_60 = data.tail(60)
    df_last_60 = df_last_60.drop(columns=['PLAYER_ID','GAME_ID','GAME_DATE','PTS','yearSeason'])
    df_season = data.loc[data['yearSeason']=='2024']
    df_season = df_season.drop(columns=['PLAYER_ID','GAME_ID','GAME_DATE','PTS','yearSeason'])
    df = data.drop(columns=['PLAYER_ID','GAME_ID','GAME_DATE','PTS','yearSeason'])
    df = df.iloc[1:]

    # Function to select top 'n' features
    def select_top_n_features(X, y, n):
        model = RandomForestClassifier()
        model.fit(X, y)

        selector = SelectFromModel(model, max_features=n, prefit=True)
        X_selected = selector.transform(X)
        return X_selected, selector

    # Modified function to evaluate models with feature selection
    def evaluate_models_with_feature_selection(df_feat, X, y, n_features, dataset_type, type_data):
        global final_model, final_model_cv_score, final_model_score, final_scoring_type, final_dataset, type_of_data, base_score, final_X, final_y, final_x_transformer

        X_selected, selector = select_top_n_features(X, y, n_features)
        scoring_method = 'precision'  # Change to 'f1_macro', 'f1_micro', or 'f1_weighted' for multi-class

        if len(df_feat) < 100:
            cv_strategy = LeaveOneOut()
        else:
            cv_strategy = KFold(n_splits=15, shuffle=True, random_state=42)

        for name, model in models.items():
            cv_scores = np.mean(cross_val_score(model, X_selected, y, cv=cv_strategy, scoring=scoring_method, n_jobs=-1))
            if name == 'Bayesian' and cv_scores == final_model_cv_score:
                cv_scores += 0.001

            if cv_scores > final_model_cv_score:
                final_model = name
                final_model_cv_score = cv_scores
                final_model_score = cv_scores - base_score
                final_scoring_type = 'loo' if len(df_feat) < 100 else 'cv'
                final_dataset = dataset_type
                type_of_data = type_data
                final_X = X_selected
                final_y = y
                final_x_transformer = selector


    # Function to evaluate models
    def evaluate_models(df_current, X, y, dataset_type, type_data, pca_transformer = None):
        global final_model, final_model_cv_score, final_model_score, final_scoring_type, final_dataset, type_of_data, base_score, final_X, final_y, final_x_transformer

        scoring_method = 'precision'  # Change to 'f1_macro', 'f1_micro', or 'f1_weighted' for multi-class

        if len(df_current) < 100:
            cv_strategy = LeaveOneOut()
        else:
            cv_strategy = KFold(n_splits=15, shuffle=True, random_state=42)

        for name, model in models.items():
            cv_scores = np.mean(cross_val_score(model, X, y, cv=cv_strategy, scoring=scoring_method, n_jobs=-1))
            if name == 'Bayesian' and cv_scores == final_model_cv_score:
                cv_scores += 0.001

            if cv_scores > final_model_cv_score:
                final_model = name
                final_model_cv_score = cv_scores
                final_model_score = cv_scores - base_score
                final_scoring_type = 'loo' if len(df_current) < 100 else 'cv'
                final_dataset = dataset_type
                type_of_data = type_data
                final_X = X
                final_y = y
                final_x_transformer = pca_transformer

                base_score = sum(df['PtsThreshold'])/ len(df) 
    final_model = ''
    final_model_cv_score = 0
    final_model_score = 0
    final_scoring_type = 'career'
    final_dataset = -999
    type_of_data = 'noraml_normal'
    final_x_transformer = None


    final_X = None
    final_y = None

    max_feat_5_pct = round(len(df)*.05)
    max_feat_10_pct = round(len(df)*.10)

    df.fillna(df.mean(), inplace=True)

    # Separating features and target
    X = df.drop('PtsThreshold', axis=1)
    y = df['PtsThreshold']

    # Standardize the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Dimensionality Reduction
    pca = PCA(n_components=0.95) # Adjust the number of components as needed
    X_pca = pca.fit_transform(X_scaled)

    # Dimensionality Reduction
    pca_n_com = PCA(n_components=max_feat_5_pct) # Adjust the number of components as needed
    X_pca_n_comp = pca_n_com.fit_transform(X_scaled)

    # Models
    models = {
        "Logistic Regression": LogisticRegression(),
        "Decision Tree": DecisionTreeClassifier(),
#         "XGBoost": XGBClassifier(),
        "SVM": SVC(),
        "Bayesian": GaussianNB(),
    }

    base_score = sum(df_last_60['PtsThreshold'])/ len(df_last_60) 

    max_feat_5_pct = round(len(df)*.05)
    max_feat_10_pct = round(len(df)*.10)

    df_last_60.fillna(df_last_60.mean(), inplace=True)

    # Separating features and target
    X = df_last_60.drop('PtsThreshold', axis=1)
    y = df_last_60['PtsThreshold']

    # Standardize the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Dimensionality Reduction
    pca = PCA(n_components=0.95) # Adjust the number of components as needed
    X_pca = pca.fit_transform(X_scaled)

    # Dimensionality Reduction
    pca_n_com = PCA(n_components=max_feat_5_pct) # Adjust the number of components as needed
    X_pca_n_comp = pca_n_com.fit_transform(X_scaled)


    # # Evaluating models without dimensionality reduction
    # print("Evaluating models without dimensionality reduction:")
    # evaluate_models(df_last_60,X_scaled, y,'60','normal_normal')

    # Evaluating models with dimensionality reduction
    # print("\nEvaluating models with dimensionality reduction:")
    evaluate_models(df_last_60,X_pca, y,'60','pca_95', pca)

    # Evaluating models with dimensionality reduction only n comp
    # print("\nEvaluating models with dimensionality reduction only n comp:")
    evaluate_models(df_last_60,X_pca_n_comp, y,'60','pca_n',pca_n_com)

    # Example of how to use the modified function
    # print("\nEvaluating models with top 5% feature selection:")
    evaluate_models_with_feature_selection(df_last_60,X_scaled, y, max_feat_5_pct,'60','selection_5')  

    # Example of how to use the modified function
    # print("\nEvaluating models with top 10% feature selection:")
    evaluate_models_with_feature_selection(df_last_60,X_scaled, y, max_feat_10_pct,'60','selection_10')  

    df_season.fillna(df_season.mean(), inplace=True)
    base_score = sum(df_season['PtsThreshold'])/ len(df_season) 

    # Separating features and target
    X = df_season.drop('PtsThreshold', axis=1)
    y = df_season['PtsThreshold']

    max_feat_5_pct = round(len(df_season)*.05)
    max_feat_10_pct = round(len(df_season)*.10)

    # Standardize the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Dimensionality Reduction
    pca = PCA(n_components=0.95) # Adjust the number of components as needed
    X_pca = pca.fit_transform(X_scaled)

    # Dimensionality Reduction
    pca_n_com = PCA(n_components=max_feat_10_pct) # Adjust the number of components as needed
    X_pca_n_comp = pca_n_com.fit_transform(X_scaled)


    # # Evaluating models without dimensionality reduction
    # print("Evaluating models without dimensionality reduction:")
    # evaluate_models(df_season,X_scaled, y, 'season','normal_normal')

    # Evaluating models with dimensionality reduction
    # print("\nEvaluating models with dimensionality reduction:",'pca_95', pca)
    evaluate_models(df_season,X_pca, y,'season','pca_95',pca)

    # Evaluating models with dimensionality reduction only n comp
    # print("\nEvaluating models with dimensionality reduction only n comp:",pca_n_com)
    evaluate_models(df_season,X_pca_n_comp, y,'season','pca_n',pca_n_com)

    # Example of how to use the modified function
    # print("\nEvaluating models with top 5% feature selection:")
    evaluate_models_with_feature_selection(df_season,X_scaled, y, max_feat_5_pct,'season','selection_5')  

    # Example of how to use the modified function
    # print("\nEvaluating models with top 10% feature selection:")
    evaluate_models_with_feature_selection(df_season,X_scaled, y, max_feat_10_pct,'season','selection_10')  
    
    base_score = sum(df_season_buffer['PtsThreshold'])/ len(df_season_buffer) 

    max_feat_5_pct = round(len(df_season_buffer)*.05)
    max_feat_10_pct = round(len(df_season_buffer)*.10)

    df_season_buffer.fillna(df_season_buffer.mean(), inplace=True)

    # Separating features and target
    X = df_season_buffer.drop('PtsThreshold', axis=1)
    y = df_season_buffer['PtsThreshold']

    # Standardize the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Dimensionality Reduction
    pca = PCA(n_components=0.95) # Adjust the number of components as needed
    X_pca = pca.fit_transform(X_scaled)

    # Dimensionality Reduction
    pca_n_com = PCA(n_components=max_feat_5_pct) # Adjust the number of components as needed
    X_pca_n_comp = pca_n_com.fit_transform(X_scaled)

    # Evaluating models with dimensionality reduction
    evaluate_models(df_season_buffer,X_pca, y,'buffer','pca_95', pca_transformer = pca)

    # Evaluating models with dimensionality reduction only n comp
    evaluate_models(df_season_buffer,X_pca_n_comp, y,'buffer','pca_n',pca_transformer = pca_n_com)

    # Example of how to use the modified function
    evaluate_models_with_feature_selection(df_season_buffer,X_scaled, y, max_feat_5_pct,'buffer','selection_5')  

    # Example of how to use the modified function
    evaluate_models_with_feature_selection(df_season_buffer,X_scaled, y, max_feat_10_pct,'buffer','selection_10') 

    if final_model == 'Logistic Regression':
        print('lr')
        best_score = 0
        tracker = 0
#         while best_score < final_model_cv_score:
        params, best_score = perform_hyper_param_tuning_bayes_lr(final_X, final_y, randint(0, 20000))
#             if tracker == 2:
#                 break
#             tracker += 1

        lr = LogisticRegression(**params).fit(final_X, final_y)

        # Get predicted probabilities for the positive class on the training set
        prob_positive_class_train = lr.predict_proba(final_X)[:, 1]

        # Find the best threshold using training set
        best_threshold = find_best_threshold(prob_positive_class_train, final_y)

        # Prediction of probabilities on the test set
        if 'SelectFromModel' in str(final_x_transformer) or 'PCA' in str(final_x_transformer):
            transformed_X_test = final_x_transformer.transform(pred)
            prob_positive_class_test = lr.predict_proba(transformed_X_test)[:, 1]
        else:
            prob_positive_class_test = lr.predict_proba(pred)[:, 1]

        # Apply the custom threshold to make final binary prediction
        final_prediction = np.where(prob_positive_class_test > best_threshold, 1, 0)

        # Assuming you are interested in the first prediction if 'pred' contains multiple samples
        prediction = final_prediction[0]
    elif final_model == 'Bayesian':
        print('nb')
        best_score = 0
        tracker = 0
#         while best_score < final_model_cv_score:
        params, best_score = perform_hyper_param_tuning_bayes_nb(final_X, final_y, randint(0, 20000))
#             if tracker == 2:
#                 break
#             tracker += 1

        nb = GaussianNB(**params).fit(final_X, final_y)
        loo = LeaveOneOut()

        # Get predicted probabilities for the positive class on the training set
        prob_positive_class_train = nb.predict_proba(final_X)[:, 1]

        # Find the best threshold using training set
        best_threshold = find_best_threshold(prob_positive_class_train, final_y)

        # Cross-validation for model accuracy
        scores = cross_val_score(nb, final_X, final_y, cv=loo, n_jobs=-1)

        # Prediction of probabilities on the test set
        if 'SelectFromModel' in str(final_x_transformer) or 'PCA' in str(final_x_transformer):
            transformed_X_test = final_x_transformer.transform(pred)
            prob_positive_class_test = nb.predict_proba(transformed_X_test)[:, 1]
        else:
            prob_positive_class_test = nb.predict_proba(pred)[:, 1]

        # Apply the custom threshold to make final binary prediction
        final_prediction = np.where(prob_positive_class_test > best_threshold, 1, 0)

        prediction = final_prediction[0]
        # Assuming you are interested in the first prediction if 'pred' contains multiple samples

    elif final_model == 'Decision Tree':
        print('dt')
        best_score = 0
        tracker = 0
#         while best_score < final_model_cv_score:
        params, best_score = perform_hyper_param_tuning_bayes_dt(final_X, final_y, randint(0, 20000))
#             if tracker == 2:
#                 break
#             tracker += 1

        dt = DecisionTreeClassifier(**params).fit(final_X, final_y)

        # Get predicted probabilities for the positive class on the training set
        prob_positive_class_train = dt.predict_proba(final_X)[:, 1]

        # Find the best threshold using training set
        best_threshold = find_best_threshold(prob_positive_class_train, final_y)

        # Prediction of probabilities on the test set
        if 'SelectFromModel' in str(final_x_transformer) or 'PCA' in str(final_x_transformer):
            transformed_X_test = final_x_transformer.transform(pred)
            prob_positive_class_test = dt.predict_proba(transformed_X_test)[:, 1]
        else:
            prob_positive_class_test = dt.predict_proba(pred)[:, 1]

        # Apply the custom threshold to make final binary prediction
        final_prediction = np.where(prob_positive_class_test > best_threshold, 1, 0)

        prediction = final_prediction[0]
        # Assuming you are interested in the first prediction if 'pred' contains multiple samples

    elif final_model == 'Random Forest':
        best_score = 0
        tracker = 0
#         while best_score < final_model_cv_score:
        params, best_score = perform_hyper_param_tuning_bayes_rf(final_X, final_y, randint(0, 20000))
#             if tracker == 2:
#                 break
#             tracker += 1

        rf = RandomForestClassifier(**params, n_jobs=-1).fit(final_X, final_y)

        # Get predicted probabilities for the positive class on the training set
        prob_positive_class_train = rf.predict_proba(final_X)[:, 1]

        # Find the best threshold using training set
        best_threshold = find_best_threshold(prob_positive_class_train, final_y)

        # Prediction of probabilities on the test set
        if 'SelectFromModel' in str(final_x_transformer) or 'PCA' in str(final_x_transformer):
            transformed_X_test = final_x_transformer.transform(pred)
            prob_positive_class_test = rf.predict_proba(transformed_X_test)[:, 1]
        else:
            prob_positive_class_test = rf.predict_proba(pred)[:, 1]

        # Apply the custom threshold to make final binary prediction
        final_prediction = np.where(prob_positive_class_test > best_threshold, 1, 0)

        prediction = final_prediction[0]
        # Assuming you are interested in the first prediction if 'pred' contains multiple samples

    elif final_model == 'XGBoost':
        print('xgb')
        best_score = 0
        tracker = 0
#         while best_score < final_model_cv_score:
        params, best_score = perform_hyper_param_tuning_bayes_xgb(final_X, final_y, randint(0, 20000))
#             if tracker == 2:
#                 break
#             tracker += 1

        xgb = XGBClassifier(**params).fit(final_X, final_y)

        # Get predicted probabilities for the positive class on the training set
        prob_positive_class_train = xgb.predict_proba(final_X)[:, 1]

        # Find the best threshold using training set
        best_threshold = find_best_threshold(prob_positive_class_train, final_y)

        # Prediction of probabilities on the test set
        if 'SelectFromModel' in str(final_x_transformer) or 'PCA' in str(final_x_transformer):
            transformed_X_test = final_x_transformer.transform(pred)
            prob_positive_class_test = xgb.predict_proba(transformed_X_test)[:, 1]
        else:
            prob_positive_class_test = xgb.predict_proba(pred)[:, 1]

        # Apply the custom threshold to make final binary prediction
        final_prediction = np.where(prob_positive_class_test > best_threshold, 1, 0)

        prediction = final_prediction[0]

    elif final_model == 'CatBoost': 
        best_score =0 
        tracker = 0
        while best_score <final_model_cv_score:
            params, best_score = perform_hyper_param_tuning_bayes_catboost(final_X,final_y,randint(0,20000))
            if tracker ==2:
                break
            tracker+=1

        cb = CatBoostClassifier(**params, verbose=False).fit(final_X,final_y)
        if 'SelectFromModel' in str(final_x_transformer) or 'PCA' in str(final_x_transformer):
            prediction = cb.predict(final_x_transformer.transform(pred))
    elif final_model == 'LightGBM':
        best_score = 0
        tracker = 0
        final_model_cv_score=.01
        while best_score < final_model_cv_score:
            params, best_score = perform_hyper_param_tuning_bayes_lgbm(final_X, final_y, randint(0, 20000))
            if tracker == 2:
                break
            tracker += 1

        lgbm = LGBMClassifier(**params, verbose=0).fit(final_X, final_y)

        # Get predicted probabilities for the positive class on the training set
        prob_positive_class_train = lgbm.predict_proba(final_X)[:, 1]

        # Find the best threshold using training set
        best_threshold = find_best_threshold(prob_positive_class_train, final_y)

        # Prediction of probabilities on the test set
        if 'SelectFromModel' in str(final_x_transformer) or 'PCA' in str(final_x_transformer):
            transformed_X_test = final_x_transformer.transform(pred)
            prob_positive_class_test = lgbm.predict_proba(transformed_X_test)[:, 1]
        else:
            prob_positive_class_test = lgbm.predict_proba(pred)[:, 1]

        # Apply the custom threshold to make final binary prediction
        final_prediction = np.where(prob_positive_class_test > best_threshold, 1, 0)

        # Assuming you are interested in the first prediction if 'pred' contains multiple samples
        prediction = final_prediction[0]
        # Assuming you are interested in the first prediction if 'pred' contains multiple samples

    elif final_model == 'SVM': 
        print('its and svm')
        best_score =0 
        tracker = 0
#         while best_score <final_model_cv_score:
        params, best_score = perform_hyper_param_tuning_bayes_svc(final_X,final_y,randint(0,20000))
#             if tracker ==2:
#                 break
#             tracker+=1
        svm = SVC(**params, probability=True, verbose=False).fit(final_X, final_y)
        prob_positive_class = svm.predict_proba(final_X)[:, 1]

        # Find the best threshold
        best_threshold = find_best_threshold(prob_positive_class, final_y)

        # Prediction of probabilities
        # Assuming final_X_test is your test set features
        if 'SelectFromModel' in str(final_x_transformer) or 'PCA' in str(final_x_transformer):
            transformed_X_test = final_x_transformer.transform(pred)
            prob_positive_class = svm.predict_proba(transformed_X_test)[:, 1]
        else:
            prob_positive_class = svm.predict_proba(pred)[:, 1]
        prob_positive_class_test = prob_positive_class
        prediction = int(prob_positive_class[0]>best_threshold)

    elif final_model == 'KNN':
        best_score = 0
        tracker = 0
#         while best_score < final_model_cv_score:
        params, best_score = perform_hyper_param_tuning_knn(final_X, final_y, randint(0, 20000))

#             # Map indices to actual values for 'metric' and 'weights'
#             params['metric'] = ['euclidean', 'manhattan', 'minkowski'][params['metric']]
#             params['weights'] = ['uniform', 'distance'][params['weights']]

#             if tracker == 2:
#                 break
#             tracker += 1

        knn = KNeighborsClassifier(**params).fit(final_X, final_y)

        if 'SelectFromModel' in str(final_x_transformer) or 'PCA' in str(final_x_transformer):
            prediction = knn.predict(final_x_transformer.transform(pred))
        prob_positive_class_test = prediction[0]
    elif final_model == 'Regularized Neural Network':
        best_score =0 
        tracker = 0
        while best_score <final_model_cv_score:
            params, best_score = perform_hyper_param_tuning_bayes_mlp(final_X,final_y,randint(0,20000))
            if tracker ==2:
                break
            tracker+=1

        mlp = MLPClassifier(**params, verbose=False).fit(final_X,final_y)
        if 'SelectFromModel' in str(final_x_transformer) or 'PCA' in str(final_x_transformer):
            prediction = mlp.predict(final_x_transformer.transform(pred))
    elif final_model == 'Bayesian':
        print('nb')
        best_score = 0
        tracker = 0
        while best_score < final_model_cv_score:
            params, best_score = perform_hyper_param_tuning_bayes_nb(final_X, final_y, randint(0, 20000))
            if tracker == 2:
                break
            tracker += 1

        nb = GaussianNB(**params).fit(final_X, final_y)
        loo = LeaveOneOut()

        # Get predicted probabilities for the positive class on the training set
        prob_positive_class_train = nb.predict_proba(final_X)[:, 1]

        # Find the best threshold using training set
        best_threshold = find_best_threshold(prob_positive_class_train, final_y)

        # Cross-validation for model accuracy
        scores = cross_val_score(nb, final_X, final_y, cv=loo, n_jobs=-1)

        # Prediction of probabilities on the test set
        if 'SelectFromModel' in str(final_x_transformer) or 'PCA' in str(final_x_transformer):
            transformed_X_test = final_x_transformer.transform(pred)
            prob_positive_class_test = nb.predict_proba(transformed_X_test)[:, 1]
        else:
            prob_positive_class_test = nb.predict_proba(pred)[:, 1]

        # Apply the custom threshold to make final binary prediction
        final_prediction = np.where(prob_positive_class_test > best_threshold, 1, 0)

        prediction = final_prediction[0]
        # Assuming you are interested in the first prediction if 'pred' contains multiple samples
    return prediction, prob_positive_class_test, best_score

In [38]:
final_model
final_model_cv_score 
final_model_score 
final_scoring_type 
final_dataset 
type_of_data 
final_x_transformer
base_score 

0

In [39]:
# model('203915',11.5,'MIN')

In [78]:
lines = pd.read_csv('betr_current_pts_lines.csv')


In [79]:
server = 'localhost\SQLEXPRESS'
atabase = 'nba_game_data'
sql = f'''
SELECT distinct
      [PLAYER]
      ,[NICKNAME]
      ,[PLAYER_SLUG]
      ,[PLAYER_ID]
  FROM [nba_game_data].[dbo].[CommonTeamRoster]
'''
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()
players = pd.read_sql(sql,cnxn)

In [80]:
lines['PLAYER_NAME'] = lines['PLAYER_NAME'].astype(str)
lines['PLAYER_NAME'] = [x.strip() for x in lines['PLAYER_NAME']]
lines['player_slug'] = [slugify(x) for x in lines['PLAYER_NAME']]

In [81]:
lines = lines.merge(players, left_on = 'player_slug', right_on='PLAYER_SLUG')

In [82]:
lines['model_guess_over'] = ''
lines['model__guess_proba_over'] = ''
lines['model_score_over'] = ''

lines['model_guess_under'] = ''
lines['model__guess_proba_under'] = ''
lines['model_score_under'] = ''

lines['data_used'] = ''
lines['model_type'] = ''

In [83]:
lines = lines.drop_duplicates()

In [84]:
lines

,PLAYER_NAME,OPP,PTS,player_slug,PLAYER,NICKNAME,PLAYER_SLUG,PLAYER_ID,model_guess_over,model__guess_proba_over,model_score_over,model_guess_under,model__guess_proba_under,model_score_under,data_used,model_type
0,Pascal Siakam,NYK,21.5,pascal-siakam,Pascal Siakam,Pascal,pascal-siakam,1627783,,,,,,,,
1,Tyrese Haliburton,NYK,15.5,tyrese-haliburton,Tyrese Haliburton,Tyrese,tyrese-haliburton,1630169,,,,,,,,
2,Aaron Nesmith,NYK,14.5,aaron-nesmith,Aaron Nesmith,Aaron,aaron-nesmith,1630174,,,,,,,,
3,Myles Turner,NYK,16.5,myles-turner,Myles Turner,Myles,myles-turner,1626167,,,,,,,,
4,Jalen Brunson,IND,32.5,jalen-brunson,Jalen Brunson,Jalen,jalen-brunson,1628973,,,,,,,,
5,Donte DiVincenzo,IND,18.5,donte-divincenzo,Donte DiVincenzo,Donte,donte-divincenzo,1628978,,,,,,,,
6,Josh Hart,IND,11.5,josh-hart,Josh Hart,Josh,josh-hart,1628404,,,,,,,,
7,Isaiah HartenStein,IND,9.5,isaiah-hartenstein,Isaiah Hartenstein,Isaiah,isaiah-hartenstein,1628392,,,,,,,,


In [85]:
# results = pd.DataFrame()
for index, row in lines.iterrows():
    print(str(row['PLAYER_NAME']),row['PTS'],str(row['OPP']))
    try:
        line = row['PTS'] + 2
        print(line)
        predict_over, prectic_proba_over, model_score_over = model(str(row['PLAYER_ID']),line,str(row['OPP']))
        print(prectic_proba_over,model_score_over)
        
        line = row['PTS']-2
        print(line)
        predict_under, prectic_proba_under, model_score_under = model(str(row['PLAYER_ID']),line,str(row['OPP']),'under')
        print(prectic_proba_under,model_score_under)
        
        if (prectic_proba_over > .5 and model_score_over > .75) or (prectic_proba_under > .5 and model_score_under > .6):
            lines.iloc[index,8] = predict_over
            lines.iloc[index,9] = prectic_proba_over[0]
            lines.iloc[index,10] = model_score_over

            lines.iloc[index,11] = predict_under
            lines.iloc[index,12] = prectic_proba_under[0]
            lines.iloc[index,13] = model_score_under

            lines.iloc[index,14] = final_dataset
            lines.iloc[index,15]=str(final_model)

    
    except:
        print(row['PLAYER_NAME'], 'failed')

Pascal Siakam 21.5 NYK
23.5
nb
100%|█████████████████████████████████████████████| 350/350 [00:55<00:00,  6.34trial/s, best loss: -0.2682926829268293]
[1.] 0.2682926829268293
19.5
dt
100%|██████████████████████████████████████████████| 100/100 [00:11<00:00,  8.34trial/s, best loss: -0.717391304347826]
[0.73333333] 0.717391304347826
Tyrese Haliburton 15.5 NYK
17.5
nb
100%|█████████████████████████████████████████████| 350/350 [00:29<00:00, 12.02trial/s, best loss: -0.7272727272727273]
[1.] 0.7272727272727273
13.5
nb
100%|████████████████████████████████████████████| 350/350 [00:28<00:00, 12.19trial/s, best loss: -0.09090909090909091]
[0.] 0.09090909090909091
Aaron Nesmith 14.5 NYK
16.5
nb
100%|████████████████████████████████████████████| 350/350 [00:50<00:00,  6.93trial/s, best loss: -0.08333333333333333]
[1.] 0.08333333333333333
12.5
its and svm
100%|█████████████████████████████████████████████| 150/150 [46:44<00:00, 18.70s/trial, best loss: -0.7195121951219512]
[0.79052346] 0.719512

In [86]:
lines

,PLAYER_NAME,OPP,PTS,player_slug,PLAYER,NICKNAME,PLAYER_SLUG,PLAYER_ID,model_guess_over,model__guess_proba_over,model_score_over,model_guess_under,model__guess_proba_under,model_score_under,data_used,model_type
0,Pascal Siakam,NYK,21.5,pascal-siakam,Pascal Siakam,Pascal,pascal-siakam,1627783,1,1.0,0.268293,1,0.733333,0.717391,season,Decision Tree
1,Tyrese Haliburton,NYK,15.5,tyrese-haliburton,Tyrese Haliburton,Tyrese,tyrese-haliburton,1630169,,,,,,,,
2,Aaron Nesmith,NYK,14.5,aaron-nesmith,Aaron Nesmith,Aaron,aaron-nesmith,1630174,1,1.0,0.083333,1,0.790523,0.719512,buffer,SVM
3,Myles Turner,NYK,16.5,myles-turner,Myles Turner,Myles,myles-turner,1626167,,,,,,,,
4,Jalen Brunson,IND,32.5,jalen-brunson,Jalen Brunson,Jalen,jalen-brunson,1628973,1,1.0,0.116667,1,0.816048,0.817073,buffer,SVM
5,Donte DiVincenzo,IND,18.5,donte-divincenzo,Donte DiVincenzo,Donte,donte-divincenzo,1628978,0,0.0,0.044444,1,1.0,0.902439,buffer,Bayesian
6,Josh Hart,IND,11.5,josh-hart,Josh Hart,Josh,josh-hart,1628404,0,0.0,0.06383,1,0.648244,0.659574,season,SVM
7,Isaiah HartenStein,IND,9.5,isaiah-hartenstein,Isaiah Hartenstein,Isaiah,isaiah-hartenstein,1628392,0,0.0,0.086957,0,0.668638,0.743902,buffer,SVM


In [87]:
lines = lines.loc[lines['model_score_over']!='']
output = lines[['PLAYER_NAME','PTS','model_guess_over','model__guess_proba_over','model_score_over','model_guess_under','model__guess_proba_under','model_score_under','model_type','data_used']]

In [88]:
output = output.sort_values(by='model_score_over', ascending= False)

In [89]:
output

,PLAYER_NAME,PTS,model_guess_over,model__guess_proba_over,model_score_over,model_guess_under,model__guess_proba_under,model_score_under,model_type,data_used
0,Pascal Siakam,21.5,1,1.0,0.268293,1,0.733333,0.717391,Decision Tree,season
4,Jalen Brunson,32.5,1,1.0,0.116667,1,0.816048,0.817073,SVM,buffer
7,Isaiah HartenStein,9.5,0,0.0,0.086957,0,0.668638,0.743902,SVM,buffer
2,Aaron Nesmith,14.5,1,1.0,0.083333,1,0.790523,0.719512,SVM,buffer
6,Josh Hart,11.5,0,0.0,0.06383,1,0.648244,0.659574,SVM,season
5,Donte DiVincenzo,18.5,0,0.0,0.044444,1,1.0,0.902439,Bayesian,buffer


In [90]:
backup = lines

In [91]:
lines.to_csv('model_outputs.csv')

In [92]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_email(subject, to_email,df):
    # Gmail account credentials
    gmail_user = 'bobbys.daily.lines@gmail.com'  # Your email
    gmail_app_password = 'zytx hudf gwky pdht'  # Your app password

     # Convert DataFrame to HTML with advanced styling
    styled_df = df.style.set_properties(**{
        'border-color': 'black',
        'border-width': '1px',
        'text-align': 'center'
    }).set_table_styles([
        {'selector': 'th',
         'props': [('background-color', '#4CAF50'),
                   ('color', 'white'),
                   ('padding', '12px')]}
    ]).set_table_attributes('style="width:100%;border-collapse:collapse;"').render()

    html = f"""\
    <html>
      <head>
        <style>
          body {{
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f3f3f3;
            color: #333;
            line-height: 1.6;
          }}
          .container {{
            width: 95%;
            margin: auto;
            overflow: hidden;
            padding: 20px 0;
          }}
          h1 {{
            font-size: 2.5em;
            margin-bottom: 10px;
          }}
          p {{
            font-size: 1.1em;
          }}
          .table-responsive {{
            overflow-x: auto;
          }}
          table {{
            width: 100%;
            margin-top: 20px;
            border-collapse: collapse;
          }}
          th, td {{
            padding: 10px;
            text-align: center;
            border: 1px solid black;
          }}
          th {{
            background-color: #4CAF50;
            color: white;
          }}
          tr:nth-child(even) {{
            background-color: #f2f2f2;
          }}
        </style>
      </head>
      <body>
        <div class="container">
          <h1>Good Morning, Mr. Plourde</h1>
          <p>Below are your daily picks for today's lines. Have a great day!</p>
          <div class="table-responsive">
            {styled_df}
          </div>
        </div>
      </body>
    </html>
    """

    # Email content setup
    msg = MIMEMultipart()
    msg['From'] = gmail_user
    msg['To'] = to_email
    msg['Subject'] = subject

    msg.attach(MIMEText(html, 'html'))
    
    # Sending email
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(gmail_user, gmail_app_password)
        server.sendmail(gmail_user, to_email, msg.as_string())
        server.quit()
        print("Email sent successfully!")
    except Exception as e:
        print(f"Error: {e}")
        print(f"Error: {e}")

# Usage
send_email(f'Daily Line Picks for {str(date.today())}', 'bobby.plourde12@yahoo.com', output)

Email sent successfully!


# alittle more forgiving 

In [93]:
lines = pd.read_csv('betr_current_pts_lines.csv')

server = 'localhost\SQLEXPRESS'
atabase = 'nba_game_data'
sql = f'''
SELECT distinct
      [PLAYER]
      ,[NICKNAME]
      ,[PLAYER_SLUG]
      ,[PLAYER_ID]
  FROM [nba_game_data].[dbo].[CommonTeamRoster]
'''
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()
players = pd.read_sql(sql,cnxn)

lines['PLAYER_NAME'] = lines['PLAYER_NAME'].astype(str)
lines['PLAYER_NAME'] = [x.strip() for x in lines['PLAYER_NAME']]
lines['player_slug'] = [slugify(x) for x in lines['PLAYER_NAME']]

lines = lines.merge(players, left_on = 'player_slug', right_on='PLAYER_SLUG')

lines['model_guess_over'] = ''
lines['model__guess_proba_over'] = ''
lines['model_score_over'] = ''

lines['model_guess_under'] = ''
lines['model__guess_proba_under'] = ''
lines['model_score_under'] = ''

lines['data_used'] = ''
lines['model_type'] = ''

# lines['PTS'] = lines['PTS']+1

lines = lines.drop_duplicates()

In [94]:
# results = pd.DataFrame()
for index, row in lines.iterrows():
    print(str(row['PLAYER_NAME']),row['PTS'],str(row['OPP']))
    try:
        line = row['PTS'] + 1
        print(line)
        predict_over, prectic_proba_over, model_score_over = model(str(row['PLAYER_ID']),line,str(row['OPP']))
        print(prectic_proba_over,model_score_over)
        
        line = row['PTS']-1
        print(line)
        predict_under, prectic_proba_under, model_score_under = model(str(row['PLAYER_ID']),line,str(row['OPP']),'under')
        print(prectic_proba_under,model_score_under)
        
        if (prectic_proba_over > .5 and model_score_over > .75) or (prectic_proba_under > .5 and model_score_under > .6):
            lines.iloc[index,8] = predict_over
            lines.iloc[index,9] = prectic_proba_over[0]
            lines.iloc[index,10] = model_score_over

            lines.iloc[index,11] = predict_under
            lines.iloc[index,12] = prectic_proba_under[0]
            lines.iloc[index,13] = model_score_under

            lines.iloc[index,14] = final_dataset
            lines.iloc[index,15]=str(final_model)

    
    except:
        print(row['PLAYER_NAME'], 'failed')

Pascal Siakam 21.5 NYK
22.5
nb
100%|█████████████████████████████████████████████| 350/350 [00:54<00:00,  6.46trial/s, best loss: -0.4268292682926829]
[0.] 0.4268292682926829
20.5
dt
100%|█████████████████████████████████████████████| 100/100 [00:12<00:00,  8.27trial/s, best loss: -0.7608695652173914]
[0.] 0.7608695652173914
Tyrese Haliburton 15.5 NYK
16.5
its and svm
100%|█████████████████████████████████████████████| 150/150 [23:53<00:00,  9.55s/trial, best loss: -0.7878787878787878]
[1.0000001e-07] 0.7878787878787878
14.5
dt
100%|█████████████████████████████████████████████| 100/100 [00:08<00:00, 12.05trial/s, best loss: -0.8787878787878788]
[0.] 0.8787878787878788
Aaron Nesmith 14.5 NYK
15.5
nb
100%|████████████████████████████████████████████| 350/350 [00:50<00:00,  6.88trial/s, best loss: -0.11666666666666667]
[1.] 0.11666666666666667
13.5
nb
100%|█████████████████████████████████████████████| 350/350 [00:53<00:00,  6.54trial/s, best loss: -0.7804878048780488]
[1.] 0.78048780487

In [95]:
lines = lines.loc[lines['model_score_over']!='']
output = lines[['PLAYER_NAME','PTS','model_guess_over','model__guess_proba_over','model_score_over','model_guess_under','model__guess_proba_under','model_score_under','model_type','data_used']]

In [96]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_email(subject, to_email,df):
    # Gmail account credentials
    gmail_user = 'bobbys.daily.lines@gmail.com'  # Your email
    gmail_app_password = 'zytx hudf gwky pdht'  # Your app password

     # Convert DataFrame to HTML with advanced styling
    styled_df = df.style.set_properties(**{
        'border-color': 'black',
        'border-width': '1px',
        'text-align': 'center'
    }).set_table_styles([
        {'selector': 'th',
         'props': [('background-color', '#4CAF50'),
                   ('color', 'white'),
                   ('padding', '12px')]}
    ]).set_table_attributes('style="width:100%;border-collapse:collapse;"').render()

    html = f"""\
    <html>
      <head>
        <style>
          body {{
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f3f3f3;
            color: #333;
            line-height: 1.6;
          }}
          .container {{
            width: 95%;
            margin: auto;
            overflow: hidden;
            padding: 20px 0;
          }}
          h1 {{
            font-size: 2.5em;
            margin-bottom: 10px;
          }}
          p {{
            font-size: 1.1em;
          }}
          .table-responsive {{
            overflow-x: auto;
          }}
          table {{
            width: 100%;
            margin-top: 20px;
            border-collapse: collapse;
          }}
          th, td {{
            padding: 10px;
            text-align: center;
            border: 1px solid black;
          }}
          th {{
            background-color: #4CAF50;
            color: white;
          }}
          tr:nth-child(even) {{
            background-color: #f2f2f2;
          }}
        </style>
      </head>
      <body>
        <div class="container">
          <h1>Good Morning, Mr. Plourde</h1>
          <p>Below are your daily picks for today's lines. Have a great day!</p>
          <div class="table-responsive">
            {styled_df}
          </div>
        </div>
      </body>
    </html>
    """

    # Email content setup
    msg = MIMEMultipart()
    msg['From'] = gmail_user
    msg['To'] = to_email
    msg['Subject'] = subject

    msg.attach(MIMEText(html, 'html'))
    
    # Sending email
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(gmail_user, gmail_app_password)
        server.sendmail(gmail_user, to_email, msg.as_string())
        server.quit()
        print("Email sent successfully!")
    except Exception as e:
        print(f"Error: {e}")
        print(f"Error: {e}")

# Usage
send_email(f'Daily Line Picks for {str(date.today())} (Non-Picky)', 'bobby.plourde12@yahoo.com', output)

Email sent successfully!


In [ ]:
time.sleep(30)
# This command will put the computer to sleep in Windows
os.system("rundll32.exe powrprof.dll,SetSuspendState 0,1,0")
